<h1>End to End Learning for Self-Driving Cars</h1>

<h4>This is a tensorFlow implementation of a Nvidia research paper with some changes by Sully Chen</h4>

In [2]:
# Credits: https://github.com/SullyChen/Autopilot-TensorFlow
# Research paper: End to End Learning for Self-Driving Cars by Nvidia. [https://arxiv.org/pdf/1604.07316.pdf]

# NVidia dataset: 72 hrs of video => 72*60*60*30 = 7,776,000 images
# Nvidia blog: https://devblogs.nvidia.com/deep-learning-self-driving-cars/

# Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
# Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB

# If you want to try on a slightly large dataset: 70 minutes of data ~ 223GB
# Refer: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5
# Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed

# Additional Installations:
# pip3 install h5py


# AWS: https://aws.amazon.com/blogs/machine-learning/get-started-with-deep-learning-using-the-aws-deep-learning-ami/

# Youtube:https://www.youtube.com/watch?v=qhUvQiKec2U
# Further reading and extensions: https://medium.com/udacity/teaching-a-machine-to-steer-a-car-d73217f2492c
# More data: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5

<h2>Reading images, Splitting data(70:30) and Creating batches for training</h2>

In [3]:
import cv2
import random
import numpy as np

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * 3.14159265 / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
c = list(zip(xs, ys))
random.shuffle(c)
xs, ys = zip(*c)

train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(cv2.resize(cv2.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], (200, 66)) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(cv2.resize(cv2.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], (200, 66)) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

In [4]:
import matplotlib.pyplot as plt

plt.hist(train_ys, bins = 50, histtype = "step")
plt.hist(val_ys, bins = 50, histtype = "step")
plt.title("Steering Wheel angle in train and val")
plt.xlabel("Angle")
plt.ylabel("Bin count")
plt.show()

<Figure size 640x480 with 1 Axes>

<h2>Model Architecture</h2>

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply((tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h2>Training the model</h2>

<h4>To visualize training using Tensorboard use tensorboard --logdir=./logs, then open http://0.0.0.0:6006/ into your web browser.</h4>

In [6]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)

# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
previous_i = 0
previous_loss = 0
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):        
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.5})
        if i % 10 == 0:            
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x: xs, y_: ys, keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x: xs, y_: ys, keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)            
            checkpoint_path = os.path.join(LOGDIR, "model_.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

sess.close()

Epoch: 0, Step: 0, Loss: 7.67311
Epoch: 0, Step: 10, Loss: 6.28109
Epoch: 0, Step: 20, Loss: 6.27818
Epoch: 0, Step: 30, Loss: 6.52348
Epoch: 0, Step: 40, Loss: 6.3585
Epoch: 0, Step: 50, Loss: 6.1843
Epoch: 0, Step: 60, Loss: 6.30853
Epoch: 0, Step: 70, Loss: 6.03263
Epoch: 0, Step: 80, Loss: 6.78401
Epoch: 0, Step: 90, Loss: 5.99166
Epoch: 0, Step: 100, Loss: 6.04139
Epoch: 0, Step: 110, Loss: 6.06215
Epoch: 0, Step: 120, Loss: 5.94563
Epoch: 0, Step: 130, Loss: 6.03258
Epoch: 0, Step: 140, Loss: 5.93112
Epoch: 0, Step: 150, Loss: 5.90773
Epoch: 0, Step: 160, Loss: 5.89275
Epoch: 0, Step: 170, Loss: 6.46352
Epoch: 0, Step: 180, Loss: 5.82709
Epoch: 0, Step: 190, Loss: 5.85735
Epoch: 0, Step: 200, Loss: 5.70376
Epoch: 0, Step: 210, Loss: 5.70414
Epoch: 0, Step: 220, Loss: 5.67082
Epoch: 0, Step: 230, Loss: 5.70979
Epoch: 0, Step: 240, Loss: 5.81898
Epoch: 0, Step: 250, Loss: 5.58706
Epoch: 0, Step: 260, Loss: 5.77134
Epoch: 0, Step: 270, Loss: 5.47172
Epoch: 0, Step: 280, Loss: 5.5727

Epoch: 5, Step: 500, Loss: 3.31576
Epoch: 5, Step: 510, Loss: 3.07297
Epoch: 5, Step: 520, Loss: 3.20025
Epoch: 5, Step: 530, Loss: 3.00351
Epoch: 5, Step: 540, Loss: 3.12413
Epoch: 5, Step: 550, Loss: 3.20275
Epoch: 5, Step: 560, Loss: 3.00029
Epoch: 5, Step: 570, Loss: 2.98228
Epoch: 5, Step: 580, Loss: 2.99457
Epoch: 5, Step: 590, Loss: 2.95601
Epoch: 5, Step: 600, Loss: 3.39504
Epoch: 5, Step: 610, Loss: 3.19497
Epoch: 5, Step: 620, Loss: 2.95373
Epoch: 5, Step: 630, Loss: 3.08154
Epoch: 5, Step: 640, Loss: 2.91167
Epoch: 5, Step: 650, Loss: 2.94201
Epoch: 5, Step: 660, Loss: 3.01919
Epoch: 5, Step: 670, Loss: 3.17704
Epoch: 5, Step: 680, Loss: 2.90317
Epoch: 5, Step: 690, Loss: 3.10311
Epoch: 5, Step: 700, Loss: 2.88832
Epoch: 5, Step: 710, Loss: 2.78534
Epoch: 5, Step: 720, Loss: 2.99917
Epoch: 5, Step: 730, Loss: 2.91863
Epoch: 5, Step: 740, Loss: 2.95535
Epoch: 5, Step: 750, Loss: 3.57327
Epoch: 5, Step: 760, Loss: 2.96997
Epoch: 5, Step: 770, Loss: 3.09727
Epoch: 5, Step: 780,

Epoch: 9, Step: 1340, Loss: 1.914
Epoch: 9, Step: 1350, Loss: 1.68085
Model saved in file: ./save\model_.ckpt
Epoch: 10, Step: 1000, Loss: 1.97046
Epoch: 10, Step: 1010, Loss: 1.81851
Epoch: 10, Step: 1020, Loss: 1.83287
Epoch: 10, Step: 1030, Loss: 2.4562
Epoch: 10, Step: 1040, Loss: 2.12509
Epoch: 10, Step: 1050, Loss: 1.97832
Epoch: 10, Step: 1060, Loss: 1.74025
Epoch: 10, Step: 1070, Loss: 1.84398
Epoch: 10, Step: 1080, Loss: 1.67518
Epoch: 10, Step: 1090, Loss: 1.79572
Epoch: 10, Step: 1100, Loss: 1.90876
Epoch: 10, Step: 1110, Loss: 1.6345
Epoch: 10, Step: 1120, Loss: 1.76336
Epoch: 10, Step: 1130, Loss: 1.75527
Epoch: 10, Step: 1140, Loss: 1.84483
Epoch: 10, Step: 1150, Loss: 1.76825
Epoch: 10, Step: 1160, Loss: 1.86234
Epoch: 10, Step: 1170, Loss: 1.63979
Epoch: 10, Step: 1180, Loss: 1.61792
Epoch: 10, Step: 1190, Loss: 1.72178
Epoch: 10, Step: 1200, Loss: 1.81262
Epoch: 10, Step: 1210, Loss: 1.63673
Epoch: 10, Step: 1220, Loss: 1.77298
Epoch: 10, Step: 1230, Loss: 1.7611
Epoch

Epoch: 14, Step: 1710, Loss: 1.05311
Epoch: 14, Step: 1720, Loss: 1.10651
Epoch: 14, Step: 1730, Loss: 1.03229
Epoch: 14, Step: 1740, Loss: 0.984082
Epoch: 14, Step: 1750, Loss: 0.952878
Epoch: 14, Step: 1760, Loss: 0.893762
Epoch: 14, Step: 1770, Loss: 0.967761
Epoch: 14, Step: 1780, Loss: 0.92539
Epoch: 14, Step: 1790, Loss: 0.920639
Epoch: 14, Step: 1800, Loss: 0.99949
Epoch: 14, Step: 1810, Loss: 1.01627
Epoch: 14, Step: 1820, Loss: 0.899519
Epoch: 14, Step: 1830, Loss: 0.967977
Epoch: 14, Step: 1840, Loss: 0.897089
Epoch: 14, Step: 1850, Loss: 1.4912
Model saved in file: ./save\model_.ckpt
Epoch: 15, Step: 1500, Loss: 0.896657
Epoch: 15, Step: 1510, Loss: 0.906862
Epoch: 15, Step: 1520, Loss: 0.938641
Epoch: 15, Step: 1530, Loss: 0.916571
Epoch: 15, Step: 1540, Loss: 0.921383
Epoch: 15, Step: 1550, Loss: 0.938619
Epoch: 15, Step: 1560, Loss: 0.891473
Epoch: 15, Step: 1570, Loss: 0.952938
Epoch: 15, Step: 1580, Loss: 1.24452
Epoch: 15, Step: 1590, Loss: 0.900749
Epoch: 15, Step: 16

Epoch: 19, Step: 2030, Loss: 0.469495
Epoch: 19, Step: 2040, Loss: 0.491235
Epoch: 19, Step: 2050, Loss: 0.495777
Epoch: 19, Step: 2060, Loss: 0.483808
Epoch: 19, Step: 2070, Loss: 0.456811
Epoch: 19, Step: 2080, Loss: 0.684246
Epoch: 19, Step: 2090, Loss: 0.472246
Epoch: 19, Step: 2100, Loss: 0.493735
Epoch: 19, Step: 2110, Loss: 0.498045
Epoch: 19, Step: 2120, Loss: 0.449045
Epoch: 19, Step: 2130, Loss: 0.472175
Epoch: 19, Step: 2140, Loss: 0.55242
Epoch: 19, Step: 2150, Loss: 0.527273
Epoch: 19, Step: 2160, Loss: 0.506962
Epoch: 19, Step: 2170, Loss: 0.46484
Epoch: 19, Step: 2180, Loss: 0.554471
Epoch: 19, Step: 2190, Loss: 0.508279
Epoch: 19, Step: 2200, Loss: 0.504748
Epoch: 19, Step: 2210, Loss: 0.45844
Epoch: 19, Step: 2220, Loss: 0.49105
Epoch: 19, Step: 2230, Loss: 0.457136
Epoch: 19, Step: 2240, Loss: 0.434424
Epoch: 19, Step: 2250, Loss: 0.447348
Epoch: 19, Step: 2260, Loss: 0.503498
Epoch: 19, Step: 2270, Loss: 0.447201
Epoch: 19, Step: 2280, Loss: 0.49401
Epoch: 19, Step: 

Epoch: 23, Step: 2720, Loss: 0.287517
Epoch: 23, Step: 2730, Loss: 0.30608
Epoch: 23, Step: 2740, Loss: 0.314339
Epoch: 23, Step: 2750, Loss: 0.292915
Model saved in file: ./save\model_.ckpt
Epoch: 24, Step: 2400, Loss: 0.323057
Epoch: 24, Step: 2410, Loss: 0.281921
Epoch: 24, Step: 2420, Loss: 0.320864
Epoch: 24, Step: 2430, Loss: 0.332671
Epoch: 24, Step: 2440, Loss: 0.375103
Epoch: 24, Step: 2450, Loss: 0.286191
Epoch: 24, Step: 2460, Loss: 0.280648
Epoch: 24, Step: 2470, Loss: 0.26738
Epoch: 24, Step: 2480, Loss: 0.35639
Epoch: 24, Step: 2490, Loss: 0.328576
Epoch: 24, Step: 2500, Loss: 0.293509
Epoch: 24, Step: 2510, Loss: 0.281087
Epoch: 24, Step: 2520, Loss: 0.37405
Epoch: 24, Step: 2530, Loss: 0.276679
Epoch: 24, Step: 2540, Loss: 0.316837
Epoch: 24, Step: 2550, Loss: 0.333364
Epoch: 24, Step: 2560, Loss: 0.31889
Epoch: 24, Step: 2570, Loss: 0.282127
Epoch: 24, Step: 2580, Loss: 0.269545
Epoch: 24, Step: 2590, Loss: 0.311662
Epoch: 24, Step: 2600, Loss: 0.319131
Epoch: 24, Step

Epoch: 28, Step: 3040, Loss: 0.232003
Epoch: 28, Step: 3050, Loss: 0.270579
Epoch: 28, Step: 3060, Loss: 0.273664
Epoch: 28, Step: 3070, Loss: 0.284457
Epoch: 28, Step: 3080, Loss: 0.227022
Epoch: 28, Step: 3090, Loss: 0.218308
Epoch: 28, Step: 3100, Loss: 0.221637
Epoch: 28, Step: 3110, Loss: 0.231093
Epoch: 28, Step: 3120, Loss: 0.221182
Epoch: 28, Step: 3130, Loss: 0.229028
Epoch: 28, Step: 3140, Loss: 0.280058
Epoch: 28, Step: 3150, Loss: 0.212511
Epoch: 28, Step: 3160, Loss: 0.229792
Epoch: 28, Step: 3170, Loss: 0.229158
Epoch: 28, Step: 3180, Loss: 0.223429
Epoch: 28, Step: 3190, Loss: 0.222374
Epoch: 28, Step: 3200, Loss: 0.232181
Epoch: 28, Step: 3210, Loss: 0.219031
Epoch: 28, Step: 3220, Loss: 0.325448
Epoch: 28, Step: 3230, Loss: 0.228704
Epoch: 28, Step: 3240, Loss: 0.219784
Epoch: 28, Step: 3250, Loss: 0.249308
Model saved in file: ./save\model_.ckpt
Epoch: 29, Step: 2900, Loss: 0.214017
Epoch: 29, Step: 2910, Loss: 0.248228
Epoch: 29, Step: 2920, Loss: 0.233669
Epoch: 29,

<h2>Run through Images and make predictions</h2>

In [ ]:
import tensorflow as tf
import scipy.misc 
import model
import cv2
from subprocess import call

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

cap = cv2.VideoCapture(0)
while(cv2.waitKey(10) != ord('q')):
    ret, frame = cap.read()
    image = scipy.misc.imresize(frame, [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 0.5})[0][0] * 180 / scipy.pi
    call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow('frame', frame)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)

cap.release()
cv2.destroyAllWindows()


<h1>Run the run_dataset.py python file to visualize the result</h1>

In [ ]:
#This is run_dataset.py. Run this file to visualize the result.

import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()
